# Emotion and Reason in Political Language: Examining the UN General speeches
## Script 1: Preprocessing & Token Frequencies
## Author: Sarah Franzen

### Instructions BEFORE running this script:
- Ensure all required packages are installed. If not, set `InstallPackages = TRUE` (see code cells below).  
- Set your working directory appropriately.  
- The script will automatically create the required folder structure.  
- Later, you will be asked to download the data from:  
  https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/0TJX8Y  
  and store it **unzipped** inside the created folder *data_original*


### Description: 
- Extract documents from their original txt documents and store them as one csv
- Data Cleaning and Pre-Processing
- Count word frequencies and weight themhemnal

## Setup, Installation and Verification of required Packages and Libraries

In [3]:
InstallPackages = False # Set this to True to install the following packages 

if InstallPackages:
    import sys

    packages = [
        "pandas",
        "nltk",
        "spacy",
        "numpy",
        "gensim",
        "pycountry",
        "wordcloud",
        "matplotlib",
        "tqdm"
    ]

    for package in packages:
        !{sys.executable} -m pip install {package}


DownloadAdditions = False # Set this to True to download these additional resources
if DownloadAdditions:
    nltk.download("punkt")
    nltk.download("averaged_perceptron_tagger")
    spacy.cli.download('en_core_web_lg')         # Download spaCy English model (large)

#########################
# Check if all packages are included
##########################

In [4]:
# == Import standard and third-party libraries for data processing, NLP, and visualization ==

import gensim
import joblib
import nltk
import os
import pandas as pd
import pycountry
import random
import re
import spacy
import time
import pickle
import numpy as np

from collections import Counter
from itertools import chain
#from matplotlib.colors import ListedColormap
from multiprocessing import Pool, freeze_support
# from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from spacy.lang.en.stop_words import STOP_WORDS as SPACY_STOPWORDS
from string import punctuation
from tqdm import tqdm
from pathlib import Path

# === Initialize NLP Tools ===

# Translator to remove punctuation
translator = str.maketrans('', '', punctuation)

# POS tagger (not used by SpaCy, but optionally available via NLTK)
tagger = nltk.perceptron.PerceptronTagger()

# Load SpaCy English model with unnecessary components disabled
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

stemmer = SnowballStemmer("english")

In [5]:
# === Set Working Directory and create folder structure ===

# Prompt user to enter working directory path
#wd = input("Please enter your working directory path (e.g., C:\\Users\\sarah\\OneDrive\\Dokumente\\Masterarbeit): ").strip()

# Change to the entered working directory
#try:
   # os.chdir(wd)
    #print(f"Working directory set to: {os.getcwd()}")
#except FileNotFoundError:
   # print("ERROR: The directory you entered does not exist. Please restart and enter a valid path.")
    #exit(1)

# Set your working directory (adjust this as needed)
wd = r"C:\Users\sarah\OneDrive\Dokumente\Masterarbeit"
os.chdir(wd)

# Set base path to current working directory
base_path = Path.cwd()
data_path = base_path / "data"

# List of subfolders to create inside 'data'
subfolders = ["data_original", "dictionaries", "freq", "preprocessed", "temp", "tokenized"]

# Create 'data' folder if it doesn't exist
data_path.mkdir(exist_ok=True)

# Create subfolders
for name in subfolders:
    (data_path / name).mkdir(exist_ok=True)

print("\nFolder structure created:")
print(f"- {data_path}")
for name in subfolders:
    print(f"  - {name}")

# Prompt user to place raw data files
#print(f"\nPlease place your raw data files (unzipped) into the folder:\n  {data_path / 'data_original'}")
#input("Press Enter after you have placed the files to continue...")

#print("Continuing with the script...")



Folder structure created:
- C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data
  - data_original
  - dictionaries
  - freq
  - preprocessed
  - temp
  - tokenized


In [6]:
# === Define Folder Paths ===

# Make sure that you have these folders in your working directory
data_c = os.path.join(wd, 'data')
data_temp = os.path.join(data_c, 'temp')
data_freq = os.path.join(data_c, 'freq')
data_dict = os.path.join(data_c, 'dictionaries')
data_preprocessed = os.path.join(data_c, 'preprocessed')
data_tokenized = os.path.join(data_c, 'tokenized')
fig = os.path.join(wd, 'fig')

In [7]:
print("Current working directory:", os.getcwd())

Current working directory: C:\Users\sarah\OneDrive\Dokumente\Masterarbeit


## Load and Prepare Corpus

### This chunk can be skipped at the moment
### Think of proper header

In [10]:
# == Load and Save Sample from UN General Debate Corpus ==

# Set Folder path containing the original TXT files    
base_folder = r".\data\data_original\UN General Debate Corpus\UNGDC_1946-2023\TXT"

#  Gather all relevant txt-files
all_txt_files = []
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith('.txt') and not file.startswith('._'):
            all_txt_files.append(os.path.join(root, file))

print(f"🧾 Total speeches found: {len(all_txt_files)}")

# Randomly pick 800 files from the full collection   ###################################################### REMOVE AT LATER POINT
sampled_files = random.sample(all_txt_files,600)

# Read the selected files into a list
raw_data = []
for filepath in sampled_files:
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
        raw_data.append({'filename': os.path.basename(filepath), 'speech': content})

# Create DataFrame from the collected speeches
df_raw = pd.DataFrame(raw_data)

df_raw = df_raw[df_raw['filename'] != '.DS_Store-to-UTF-8.txt'].copy()

# Save df_raw as a pickle file for quick future loading
raw_pickle_path = r".\data\un_corpus_raw.pkl"
df_raw.to_pickle(raw_pickle_path)

# Export df as CSV 
raw_output_path = r".\data\un_corpus_raw.csv"
df_raw.to_csv(raw_output_path, index=False, sep=';', encoding='utf-8')

print(f"\n✅ Saved raw data with {len(df_raw)} speeches to '{raw_output_path}'")


🧾 Total speeches found: 10761

✅ Saved raw data with 600 speeches to '.\data\un_corpus_raw.csv'


In [11]:
# == Check if everything worked & drop empty speeches ==

# Load df_raw
df_raw = pd.read_pickle(r".\data\un_corpus_raw.pkl")

# Drop empty speeches
df_raw['speech'] = df_raw['speech'].astype(str)
df_raw = df_raw[df_raw['speech'].str.strip() != ''].copy()

# View df to check structure
df_raw.head()         


,filename,speech
0,GIN_77_2022.txt,"At the outset of my remarks, I wish to convey ..."
1,GHA_28_1973.txt,"﻿29.\t Permit me, Sir, on behalf of the Govern..."
2,GRC_23_1968.txt,"40. Mr. President, on behalf of the Greek dele..."
3,BGR_21_1966.txt,106. On behalf of the delegation of the Pe...
4,ESP_27_1972.txt,"Mr. President, before addressing this twenty- ..."


## Create new variables

### Year, country_code and country_name

In [15]:
# Extract country code (first 3 letters) and year (last 4 digits before .txt)
df_raw['country_code'] = df_raw['filename'].str.extract(r'^([A-Z]{2,3})')
df_raw['year'] = df_raw['filename'].str.extract(r'_(\d{4})\.txt$').astype(int)

print("Min year:", df_raw['year'].min())
print("Max year:", df_raw['year'].max())

# Match country codes to country names
code_to_name = {country.alpha_3: country.name for country in pycountry.countries}

# Add custom short names and legacy codes
custom_names = {
    "BOL": "Bolivia",
    "COD": "The Democratic Republic of the Congo",
    "IRN": "Iran",
    "LAO": "Laos",
    "MDA": "Moldova",
    "PRK": "North Korea",
    "PSE": "Palestine",
    "RUS": "Russia",
    "SYR": "Syria",
    "TZA": "Tanzania",
    "VAT": "Vatican City State",
    "VEN": "Venezuela",
    "VNM": "Vietnam",
    "YMD": "South Yemen",
    "YUG": "Yugoslavia",
    "DDR": "East Germany",
    # "SUN": "Soviet Union",
    "EU": "European Union",
    "CSK": "Czechoslovakia",
    "FSM": "Micronesia",
    "KOR": "South Korea"
    
}

# Update the main mapping with custom names
code_to_name.update(custom_names)

# Map with updated dictionary
df_raw['country_name'] = df_raw['country_code'].map(code_to_name)

# Check missing mappings
missing = df_raw.loc[df_raw['country_name'].isna(), 'country_code'].unique()
print("Missing codes:", missing)

# Check structure of the df
df_raw.head() 

save_path = os.path.join(data_c, 'un_corpus_raw.pkl')
df_raw.to_pickle(save_path)
print(f"df_raw saved to {save_path}")

Min year: 1946
Max year: 2023
Missing codes: []
df_raw saved to C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\un_corpus_raw.pkl


In [16]:
# == Check the country names 

pd.set_option('display.max_rows', None)
print(df_raw[['country_code', 'country_name']].drop_duplicates().sort_values('country_code').reset_index(drop=True))
# Reset to default afterward
pd.reset_option('display.max_rows')

    country_code                          country_name
0            AFG                           Afghanistan
1            AGO                                Angola
2            ALB                               Albania
3            ARE                  United Arab Emirates
4            ARG                             Argentina
5            ARM                               Armenia
6            ATG                   Antigua and Barbuda
7            AUS                             Australia
8            AUT                               Austria
9            AZE                            Azerbaijan
10           BDI                               Burundi
11           BEL                               Belgium
12           BEN                                 Benin
13           BFA                          Burkina Faso
14           BGD                            Bangladesh
15           BGR                              Bulgaria
16           BHR                               Bahrain
17        

### Length of raw speeches

In [18]:
# Add a new column: speech length in words
df_raw['speech_length_words'] = df_raw['speech'].apply(lambda x: len(str(x).split()))

In [19]:
# Calculate average length
avg_length = df_raw['speech_length_words'].mean()

# Print it
print("Average speech length (words):", round(avg_length, 2))

# 20 shortest speeches
print("20 shortest speeches:")
print(df_raw.nsmallest(20, 'speech_length_words')[['filename', 'country_name', 'year', 'speech_length_words']])

# 20 longest speeches
print("\n20 longest speeches:")
print(df_raw.nlargest(20, 'speech_length_words')[['filename', 'country_name', 'year', 'speech_length_words']])

Average speech length (words): 3030.47
20 shortest speeches:
            filename        country_name  year  speech_length_words
146  SYC_52_1997.txt          Seychelles  1997                  597
147  IRN_03_1948.txt                Iran  1948                  813
145  OMN_66_2011.txt                Oman  2011                  892
12   GNB_78_2023.txt       Guinea-Bissau  2023                  904
584  UGA_72_2017.txt              Uganda  2017                  910
562  FIN_69_2014.txt             Finland  2014                  991
144  ERI_65_2010.txt             Eritrea  2010                  997
67   JOR_74_2019.txt              Jordan  2019                 1010
104  BOL_60_2005.txt             Bolivia  2005                 1060
83   FRA_61_2006.txt              France  2006                 1069
468  BGR_57_2002.txt            Bulgaria  2002                 1097
48   COG_71_2016.txt               Congo  2016                 1122
10   MLI_64_2009.txt                Mali  2009         

### New variable: English as Official Language

In [21]:
# Source for english as official language : https://gradschool.utk.edu/future-students/office-of-graduate-admissions/applying-to-graduate-school/admission-requirements/testing-requirements/countries-with-english-as-official-language/
# They are quoting: https://www.cia.gov/the-world-factbook/field/languages/

english_countries = [
    "Anguilla", "Antigua and Barbuda", "Bahamas", "Barbados", "Belize", "Belgium",
    "Bermuda", "Botswana", "British Virgin Islands", "Burundi", "Cameroon", "Canada",
    "Cayman Islands", "Christmas Island", "Cook Islands", "Dominica", "Fiji", "Gambia",
    "Ghana", "Grenada", "Guyana", "Hong Kong", "India", "Ireland", "Jersey", "Kenya",
    "Liberia", "Malawi", "Malta", "Marshall Islands", "Micronesia",
    "Namibia", "New Zealand", "Nigeria", "Niue", "Norfolk Island", "Northern Mariana Islands",
    "Pakistan", "Palau", "Papua New Guinea", "Philippines", "Pitcairn Islands", "Rwanda",
    "Saint Kitts and Nevis", "Saint Lucia", "Samoa", "Seychelles", "Sierra Leone", "Singapore",
    "Sint Maarten", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Sudan",
    "Eswatini", "Tanzania", "Tonga", "Trinidad and Tobago", "Turks and Caicos Islands",
    "Tuvalu", "Uganda", "Zambia", "Zimbabwe"
]

# 2. Create dummy column
df_raw['english_official_language'] = df_raw['country_name'].apply(
    lambda x: 1 if x in english_countries else 0
)

# 3. Find countries in the list that did not match any entry in df_raw
matched = set(df_raw['country_name'])
unmatched = [country for country in english_countries if country not in matched]

# 4. Print unmatched country names
print("Countries not matched in df_raw['country_name']:")
for country in unmatched:
    print(country)

# All of these countries are either British Overseas Territories, Australian Territories, self-governing island territories or Special Administrative Regions
    # None of the unmatched regions are UN Members


Countries not matched in df_raw['country_name']:
Anguilla
Bermuda
British Virgin Islands
Cameroon
Cayman Islands
Christmas Island
Cook Islands
Dominica
Hong Kong
Jersey
Malta
Micronesia
Niue
Norfolk Island
Northern Mariana Islands
Palau
Pitcairn Islands
Sint Maarten
Solomon Islands
South Sudan
Tonga
Turks and Caicos Islands


In [22]:
df_raw.head()

,filename,speech,country_code,year,country_name,speech_length_words,english_official_language
0,GIN_77_2022.txt,"At the outset of my remarks, I wish to convey ...",GIN,2022,Guinea,1794,0
1,GHA_28_1973.txt,"﻿29.\t Permit me, Sir, on behalf of the Govern...",GHA,1973,Ghana,6227,1
2,GRC_23_1968.txt,"40. Mr. President, on behalf of the Greek dele...",GRC,1968,Greece,2878,0
3,BGR_21_1966.txt,106. On behalf of the delegation of the Pe...,BGR,1966,Bulgaria,4173,0
4,ESP_27_1972.txt,"Mr. President, before addressing this twenty- ...",ESP,1972,Spain,3013,0


### New variable: permanent member security council

In [24]:
# Define permanent members of the UN Security Council
permanent_members = ['RUS', 'USA', 'FRA', 'GBR', 'CHN']

# Create dummy variable
df_raw['security_council_permanent'] = df_raw['country_code'].isin(permanent_members).astype(int)


In [25]:
print(df_raw[df_raw['country_code'].isin(permanent_members)][
    ['country_code', 'country_name', 'security_council_permanent', 'year']
])

    country_code    country_name  security_council_permanent  year
83           FRA          France                           1  2006
91           USA   United States                           1  2011
106          USA   United States                           1  1960
112          USA   United States                           1  2007
138          RUS          Russia                           1  1967
166          FRA          France                           1  2000
181          FRA          France                           1  1946
209          RUS          Russia                           1  2021
219          CHN           China                           1  2009
251          RUS          Russia                           1  1966
258          CHN           China                           1  1996
262          RUS          Russia                           1  1986
440          RUS          Russia                           1  2003
447          USA   United States                           1  

### New variables: speaker & position & gender

In [27]:
os.chdir(data_c)
df_speakers = pd.read_excel(r"data_original\UN General Debate Corpus\Speakers_by_session.xlsx")

In [28]:
df_speakers.head()

,Year,Session,ISO Code,Country,Name of Person Speaking,Post,Unnamed: 6
0,2023,78,BRA,Brazil,Luiz Inacio Lula da Silva,President,NaN
1,2023,78,USA,United States of America,Joseph R. Biden,President,NaN
2,2023,78,COL,Colombia,Gustavo Petro Urrego,President,NaN
3,2023,78,JOR,Jordan,Abdullah II ibn Al Hussein,King,NaN
4,2023,78,POL,Poland,Andrzej Duda,President,NaN


In [29]:
df_raw.head()

,filename,speech,country_code,year,country_name,speech_length_words,english_official_language,security_council_permanent
0,GIN_77_2022.txt,"At the outset of my remarks, I wish to convey ...",GIN,2022,Guinea,1794,0,0
1,GHA_28_1973.txt,"﻿29.\t Permit me, Sir, on behalf of the Govern...",GHA,1973,Ghana,6227,1,0
2,GRC_23_1968.txt,"40. Mr. President, on behalf of the Greek dele...",GRC,1968,Greece,2878,0,0
3,BGR_21_1966.txt,106. On behalf of the delegation of the Pe...,BGR,1966,Bulgaria,4173,0,0
4,ESP_27_1972.txt,"Mr. President, before addressing this twenty- ...",ESP,1972,Spain,3013,0,0


In [30]:
print(df_raw[(df_raw['country_code'] == 'MEX') & (df_raw['year'] == 1982)])


Empty DataFrame
Columns: [filename, speech, country_code, year, country_name, speech_length_words, english_official_language, security_council_permanent]
Index: []


In [31]:
df_merged = df_raw.merge(
    df_speakers[['Year', 'ISO Code', 'Name of Person Speaking', 'Post']],
    left_on=['year', 'country_code'],
    right_on=['Year', 'ISO Code'],
    how='left',
    indicator=True)

df_merged.head()


,filename,speech,country_code,year,country_name,speech_length_words,english_official_language,security_council_permanent,Year,ISO Code,Name of Person Speaking,Post,_merge
0,GIN_77_2022.txt,"At the outset of my remarks, I wish to convey ...",GIN,2022,Guinea,1794,0,0,2022.0,GIN,Bernard Gomou,Prime Minister,both
1,GHA_28_1973.txt,"﻿29.\t Permit me, Sir, on behalf of the Govern...",GHA,1973,Ghana,6227,1,0,1973.0,GHA,Baah,NaN,both
2,GRC_23_1968.txt,"40. Mr. President, on behalf of the Greek dele...",GRC,1968,Greece,2878,0,0,1968.0,GRC,Mr. PIPINELIS,NaN,both
3,BGR_21_1966.txt,106. On behalf of the delegation of the Pe...,BGR,1966,Bulgaria,4173,0,0,1966.0,BGR,Mr. BASHEV,NaN,both
4,ESP_27_1972.txt,"Mr. President, before addressing this twenty- ...",ESP,1972,Spain,3013,0,0,1972.0,ESP,Mr. Lopez-Bravo,NaN,both


In [32]:
# Merge with indicator and set unmatched rows to NA
df_merged = df_raw.merge(
    df_speakers[['Year', 'ISO Code', 'Name of Person Speaking', 'Post']],
    left_on=['year', 'country_code'],
    right_on=['Year', 'ISO Code'],
    how='left',
    indicator=True)

# Get rows with no match in df_speakers
unmatched = df_merged[df_merged['_merge'] == 'left_only']

# Print unmatched rows with selected columns (panda sets them to NA by default)
print(unmatched[['filename', 'year', 'country_code', 'country_name']])


# Drop the '_merge' column from merged df
df_merged = df_merged.drop(columns=['Year', 'ISO Code', '_merge'])

# Rename columns
df_merged = df_merged.rename(columns={
    'Name of Person Speaking': 'speaker_name',
    'Post': 'position'})

            filename  year country_code country_name
62   HTI_18_1963.txt  1963          HTI        Haiti
78   MYS_18_1963.txt  1963          MYS     Malaysia
97   EGY_17_1962.txt  1962          EGY        Egypt
299  YMD_44_1989.txt  1989          YMD  South Yemen
455  YMD_35_1980.txt  1980          YMD  South Yemen
540  SDN_18_1963.txt  1963          SDN        Sudan


In [33]:
# Create the gender_dummy 
df_merged['gender_dummy'] = df_merged['speaker_name'].apply(
    lambda name: 0 if pd.notnull(name) and re.search(r'^(?:Mr|Sir)\b', name, re.IGNORECASE)
    else 1 if pd.notnull(name) and re.search(r'^(?:Mrs|Ms)\b', name, re.IGNORECASE)
    else None
)

In [34]:
# Count all values including NaN
counts = df_merged['gender_dummy'].value_counts(dropna=False)

# Build summary using .get() to handle missing keys
gender_summary = pd.DataFrame({
    'gender_dummy': ['0 (male)', '1 (female)', 'NaN (unknown)'],
    'count': [
        counts.get(0, 0),
        counts.get(1, 0),
        counts.get(np.nan, 0)
    ]
})

print(gender_summary)

    gender_dummy  count
0       0 (male)    256
1     1 (female)      9
2  NaN (unknown)    335


In [35]:
df_merged.head()

,filename,speech,country_code,year,country_name,speech_length_words,english_official_language,security_council_permanent,speaker_name,position,gender_dummy
0,GIN_77_2022.txt,"At the outset of my remarks, I wish to convey ...",GIN,2022,Guinea,1794,0,0,Bernard Gomou,Prime Minister,NaN
1,GHA_28_1973.txt,"﻿29.\t Permit me, Sir, on behalf of the Govern...",GHA,1973,Ghana,6227,1,0,Baah,NaN,NaN
2,GRC_23_1968.txt,"40. Mr. President, on behalf of the Greek dele...",GRC,1968,Greece,2878,0,0,Mr. PIPINELIS,NaN,0.0
3,BGR_21_1966.txt,106. On behalf of the delegation of the Pe...,BGR,1966,Bulgaria,4173,0,0,Mr. BASHEV,NaN,0.0
4,ESP_27_1972.txt,"Mr. President, before addressing this twenty- ...",ESP,1972,Spain,3013,0,0,Mr. Lopez-Bravo,NaN,0.0


In [36]:
# Store df_merged

os.chdir(wd)

# Save df_merged as a pickle file for quick future loading
merged_pickle_path = r".\data\un_corpus_merged.pkl"
df_merged.to_pickle(merged_pickle_path)

# Export df as CSV 
merged_output_path = r".\data\un_corpus_merged.csv"
df_merged.to_csv(merged_output_path, index=False, sep=';', encoding='utf-8')

## Pre-processing

### Cleaning

In [39]:
# == Clean text by removing empty spaces, line breaks, hyphenation, stray characters, and escape quote ==

# Define cleaning function
def clean_text(content):
    if pd.isna(content):
        return ""
    
    # Remove line breaks and carriage returns
    content = content.replace('\n', ' ').replace('\r', ' ')

    # Collapse multiple spaces
    content = ' '.join(content.split())

    # Ensure spacing after punctuation
    content = re.sub(r'(?<=[.,])(?=[^\s])', r' ', content)

    # Remove hyphenation at line breaks (e.g. "inter- national" → "international")
    content = re.sub(r'-\s', '', content)

     # Replace hyphen between letters with a space to prevent merging words (e.g. "russian-and" → "russian and")
    content = re.sub(r'(?<=\w)-(?=\w)', ' ', content)

    # Remove stray backslashes
    content = content.replace("\\", "")

    return content

# Apply cleaning to each speech
df_raw['speech'] = df_raw['speech'].astype(str)  # Ensure column is string type
df_clean = df_raw.copy()
df_clean['speech'] = df_clean['speech'].apply(clean_text)

# Drop rows with empty speeches after cleaning
df_clean = df_clean[df_clean['speech'].str.strip().astype(bool)].reset_index(drop=True)


In [72]:
# == Split cleaned data into chunks and save as separate files ==

# Convert cleaned DataFrame to list of lists
clean_data = df_clean[['filename', 'speech']].values.tolist()

# Split cleaned data into 4 equal chunks
data_id1 = clean_data[:int(len(clean_data)/4)]
data_id2 = clean_data[int(len(clean_data)/4): int(2*len(clean_data)/4)]
data_id3 = clean_data[int(2*len(clean_data)/4): int(3*len(clean_data)/4)]
data_id4 = clean_data[int(3*len(clean_data)/4):]

# Change directory to the temp folder
os.chdir(data_temp)  # make sure `data_temp` exists and is defined

# Save each chunk with joblib
joblib.dump(data_id1, 'clean_speeches_indexed1.pkl')
joblib.dump(data_id2, 'clean_speeches_indexed2.pkl')
joblib.dump(data_id3, 'clean_speeches_indexed3.pkl')
joblib.dump(data_id4, 'clean_speeches_indexed4.pkl')

# Store list of cleaned data chunk paths to feed into preprocessing function later
clean_files = [
    os.path.join(data_temp, 'clean_speeches_indexed1.pkl'),
    os.path.join(data_temp, 'clean_speeches_indexed2.pkl'),
    os.path.join(data_temp, 'clean_speeches_indexed3.pkl'),
    os.path.join(data_temp, 'clean_speeches_indexed4.pkl')
]

print(f"✅ Saved clean speeches chunks in '{data_temp}'")

✅ Saved clean speeches chunks in 'C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp'


### Advanced Text Pre-Processing

#### Extend stopwords list

In [103]:
# Get SpaCy stopwords
SPACY_STOPWORDS = list(nlp.Defaults.stop_words)

# Path to save
stopwords_path = os.path.join(data_c, "stopwords.pkl")

# Save stopwords
joblib.dump(SPACY_STOPWORDS, stopwords_path)

print(f"Saved {len(SPACY_STOPWORDS)} stopwords to {stopwords_path}")

Saved 326 stopwords to C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\stopwords.pkl


In [75]:
# == Define function to Tokenize, eliminate digits, remove stopwords, lemmatize, POS-Tagging ==

def pro1(lista):
    # Remove punctuation
    return [[row[0], row[1].translate(translator)] for row in lista]

def pro2(lista):
    # Tokenize and lowercase with gensim
    return [[row[0], gensim.utils.simple_preprocess(row[1])] for row in lista]

def pro3(lista):
    # Remove tokens that are only digits
        a = [[row[0], [w for w in row[1] if not w.isdigit()]] for row in lista]
        return a
    
def pro4(lista):
    # Drop short words
    return [[row[0], [w for w in row[1] if len(w) > 2]] for row in lista]

def tags_spacy(lista):
    texts = [' '.join(row[1]) for row in lista]
    docs = list(nlp.pipe(texts, batch_size=20, n_process=1))
    result = []
    for i, doc in enumerate(docs):
        filtered_tokens = [token.text for token in doc if token.tag_.startswith(('N', 'V', 'J'))]
        result.append([lista[i][0], filtered_tokens])
    return result


def pro5(lista):
    # Remove stopwords using SpaCy stopword list
    return [[row[0], [w for w in row[1] if w not in SPACY_STOPWORDS]] for row in lista]

def pro6(lista):
      return [
        [row[0], [stemmer.stem(token) for token in row[1]]]
        for row in lista
    ]
########################## Question for Max: They removed procedural words in the paper

def dropnull(lista):
    # Drop empty speeches
    return [row for row in lista if len(' '.join(row[1])) > 0]

In [77]:
# == Create full pre-processing function and call it

def preprocessing(data_name):
    t0 = time.time()
    print(f"Starting preprocessing for {data_name}...")

    data = joblib.load(data_name)
    data = pro1(data)
    data = pro2(data)
    data = pro3(data)
    data = pro4(data)

    print(f"[{data_name}] Before tagging: {time.time() - t0:.2f}s")
    data = tags_spacy(data)
    print(f"[{data_name}] After tagging: {time.time() - t0:.2f}s")

    data = pro5(data)
   # data = pro6(data)
    data = dropnull(data)

    # out_name = data_name.replace('cleanspeeches_', 'preprocessed_speeches_').replace('.pkl', '_temp.pkl')

    # Store preprocessed corupus (before stemming) for wordcloud
    filename_wordcloud = data_name.replace('clean_speeches_', 'wordcloud_speeches_').replace('.pkl', '.pkl')
    out_name_wordcloud = os.path.join(data_preprocessed, os.path.basename(filename_wordcloud))
    joblib.dump(data, out_name_wordcloud)

    # Apply stemming
    data_stemmed = pro6(data)

    filename_preprocessed = data_name.replace('clean_speeches_', 'preprocessed_speeches_').replace('.pkl', '.pkl')
    out_preprocessed = os.path.join(data_preprocessed, os.path.basename(filename_preprocessed))
    joblib.dump(data_stemmed, out_preprocessed
               )
    print(f"[{data_name}] Saved stemmed version: {out_preprocessed}")

    print(f"[{data_name}] Done. Total time: {time.time() - t0:.2f}s\n")

def main():
    for fname in clean_files:
        preprocessing(fname)

if __name__ == "__main__":
    main()

Starting preprocessing for C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl...
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl] Before tagging: 1.10s
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl] After tagging: 31.77s
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl] Saved stemmed version: C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\preprocessed\preprocessed_speeches_indexed1.pkl
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed1.pkl] Done. Total time: 37.41s

Starting preprocessing for C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed2.pkl...
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed2.pkl] Before tagging: 1.04s
[C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp\clean_speeches_indexed2.pkl] After tagging: 30.06s
[C:\Users\s

In [79]:
# Store the pre-processed data
preprocessed_files = [
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed1.pkl'),
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed2.pkl'),
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed3.pkl'),
    os.path.join(data_preprocessed, 'preprocessed_speeches_indexed4.pkl')
]

wordcloud_files = [
    os.path.join(data_preprocessed, 'wordcloud_speeches_indexed1.pkl'),
    os.path.join(data_preprocessed, 'wordcloud_speeches_indexed2.pkl'),
    os.path.join(data_preprocessed, 'wordcloud_speeches_indexed3.pkl'),
    os.path.join(data_preprocessed, 'wordcloud_speeches_indexed4.pkl')
]

In [81]:
print(os.getcwd())

C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\temp


## Word-Frequencies

### Count frequencies of all tokens and display the most common words

In [85]:
#== Count token frequencies ==

def count_frequencies(filenames):
    total_freq = Counter()
    for fname in tqdm(filenames):
        data = joblib.load(fname)
        tokens = chain.from_iterable(row[1] for row in data if isinstance(row[1], list))
        total_freq.update(tokens)
    return total_freq

def remove_rare_words(filenames, freqs, min_count=10):
    for fname in filenames:
        data = joblib.load(fname)
        filtered_data = []
        for doc_id, tokens in data:
            filtered_tokens = [w for w in tokens if freqs.get(w, 0) >= min_count]
            filtered_data.append([doc_id, filtered_tokens])
        joblib.dump(filtered_data, fname)  # overwrite or save as new file
        print(f"Processed {fname}: removed words with freq < {min_count}")

# === Count for preprocessed (stemmed) speeches ===
word_counts_stemmed = count_frequencies(preprocessed_files)

remove_rare_words(preprocessed_files, word_counts_stemmed, min_count=10)

print("\n[Stemmed] Top 50 most common words:")
for word, count in word_counts_stemmed.most_common(50):
    print(f"{word}: {count}")

print("\n[Stemmed] Top 50 least common words:")
for word, count in word_counts_stemmed.most_common()[-50:]:
    print(f"{word}: {count}")

# Save stemmed word counts
save_path_stemmed = os.path.join(data_freq, 'word_counts_stemmed.pkl')
joblib.dump(word_counts_stemmed, save_path_stemmed)

# === Count for wordcloud (unstemmed) speeches ===
word_counts_wordcloud = count_frequencies(wordcloud_files)

print("\n[Wordcloud] Top 50 most common words:")
for word, count in word_counts_wordcloud.most_common(50):
    print(f"{word}: {count}")

print("\n[Wordcloud] Top 50 least common words:")
for word, count in word_counts_wordcloud.most_common()[-50:]:
    print(f"{word}: {count}")

# Save unstemmed word counts
save_path_wordcloud = os.path.join(data_freq, 'word_counts_wordcloud.pkl')
joblib.dump(word_counts_wordcloud, save_path_wordcloud)


100%|██████████| 4/4 [00:02<00:00,  1.83it/s]


Processed C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\preprocessed\preprocessed_speeches_indexed1.pkl: removed words with freq < 10
Processed C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\preprocessed\preprocessed_speeches_indexed2.pkl: removed words with freq < 10
Processed C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\preprocessed\preprocessed_speeches_indexed3.pkl: removed words with freq < 10
Processed C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data\preprocessed\preprocessed_speeches_indexed4.pkl: removed words with freq < 10

[Stemmed] Top 50 most common words:
nation: 12923
unit: 10584
countri: 10339
intern: 9114
develop: 8329
state: 7886
world: 7620
peac: 7445
peopl: 7201
secur: 4740
general: 4582
govern: 4426
econom: 4345
organ: 3835
assembl: 3768
right: 3725
year: 3677
problem: 3446
new: 3404
effort: 3276
continu: 3183
human: 3091
support: 3077
polit: 2861
communiti: 2847
time: 2685
region: 2632
member: 2513
session: 2478
africa: 2466
import: 2327
war: 

100%|██████████| 4/4 [00:02<00:00,  1.78it/s]


[Wordcloud] Top 50 most common words:
united: 10469
nations: 10232
international: 8746
world: 7458
countries: 6812
peace: 6071
states: 5709
development: 4755
people: 4585
general: 4468
security: 4398
economic: 4318
assembly: 3700
new: 3404
country: 3392
government: 3355
organization: 2874
political: 2748
community: 2664
human: 2663
efforts: 2649
peoples: 2611
africa: 2389
rights: 2367
session: 2363
support: 2272
council: 2234
time: 2204
war: 2041
south: 2033
problems: 1997
state: 1952
republic: 1914
situation: 1887
great: 1869
national: 1864
developing: 1858
years: 1854
nuclear: 1843
order: 1839
year: 1817
conference: 1748
global: 1744
social: 1717
work: 1675
relations: 1637
charter: 1569
hope: 1560
continue: 1548
african: 1547

[Wordcloud] Top 50 least common words:
irishmen: 1
absolutes: 1
cakobau: 1
wings: 1
venturing: 1
nought: 1
copra: 1
perfectionism: 1
scribed: 1
dissimilarities: 1
islanders: 1
canberra: 1
expired: 1
exhilarating: 1
transient: 1
oligarchical: 1
depolarization: 

['C:\\Users\\sarah\\OneDrive\\Dokumente\\Masterarbeit\\data\\freq\\word_counts_wordcloud.pkl']

In [86]:
os.chdir(data_c)
print(os.getcwd())

C:\Users\sarah\OneDrive\Dokumente\Masterarbeit\data


### Count the frequency of the dictionary words

In [88]:
# == Count dictionary words

# Load dictionaries            ##### How did they come up with this dictionary? Why did they exclude words?

affect_path = os.path.join(data_dict, 'dictionary_affect.pkl')
cognition_path = os.path.join(data_dict, 'dictionary_cognition.pkl')

with open(affect_path, 'rb') as f:
    affect_dict = pickle.load(f)
print("Contents of affect dictionary:")
print(affect_dict)
print("Number of words in affect dictionary:", len(affect_dict))

with open(cognition_path, 'rb') as f:
    cognition_dict = pickle.load(f)
print("Contents of cognition dictionary:")
print(cognition_dict)
print("Number of words in cognition dictionary:", len(cognition_dict))

affect = joblib.load(affect_path)
cognition = joblib.load(cognition_path)

a = [[i, word_counts_stemmed[i]] for i in affect if i in word_counts_stemmed]
c = [[i, word_counts_stemmed[i]] for i in cognition if i in word_counts_stemmed]

a = sorted(a, key=lambda x: x[1], reverse=True)
c = sorted(c, key=lambda x: x[1], reverse=True)

a = [[i[0], f"({i[1]}),"] for i in a]
c = [[i[0], f"({i[1]}),"] for i in c]

a1 = ' '.join(str(r) for v in a for r in v)
c1 = ' '.join(str(r) for v in c for r in v)

affect_out_path = os.path.join(data_freq, "affect_words.txt")
cog_out_path = os.path.join(data_freq, "cog_words.txt")

os.makedirs(data_freq, exist_ok=True)  # ensure directory exists

with open(affect_out_path, "w") as output:
    output.write(a1)

with open(cog_out_path, "w") as output:
    output.write(c1)

Contents of affect dictionary:
['forbid', 'unattract', 'cruelti', 'crappi', 'apathi', 'scari', 'unimpress', 'sin', 'dumbest', 'eas', 'agit', 'sob', 'shocker', 'tragedi', 'fabul', 'strongest', 'giver', 'sigh', 'aw', 'witch', 'hurtl', 'fucktard', 'cruel', 'glamor', 'funni', 'smarter', 'brillianc', 'irrate', 'alright', 'honest', 'profit', 'fearless', 'grievous', 'relax', 'isolationist', 'hah', 'shyness', 'poorest', 'cruelest', 'troublemak', 'disagre', 'agon', 'terror', 'fight', 'pleas', 'poor', 'crazi', 'hostil', 'stupid', 'damnat', 'vain', 'jade', 'heartless', 'nag', 'gloomi', 'damn', 'dishearten', 'pleaser', 'credit', 'warmth', 'greatest', 'whine', 'shame', 'angriest', 'envious', 'grin', 'blameless', 'sweeter', 'laidback', 'stupidest', 'unprotect', 'whiner', 'unlov', 'shake', 'boredom', 'fairer', 'weaker', 'wellb', 'bold', 'sucki', 'unsuccess', 'mourner', 'liken', 'defens', 'invigor', 'tedious', 'paranoid', 'cynic', 'dignifi', 'paranoia', 'sweetest', 'contented', 'humili', 'crush', 'ter

In [89]:
# == Calculate weighted frequencies for all words

# - downweights very common words by giving more importance to rare ones
word_counts_stemmed = joblib.load(os.path.join(data_freq, 'word_counts_stemmed.pkl'))

l = sum(word_counts_stemmed.values())

a = 0.001
word_counts_weighted = {k: a / (a + (v / l)) for k, v in word_counts_stemmed.items()}
#for key in word_counts.keys():
 #   word_counts[key] = a / (a + (word_counts[key] / l))

joblib.dump(word_counts_weighted, os.path.join(data_freq, 'word_counts_weighted.pkl'))

################################################################################ ISSUE##################
# To print top 100 by weighted values, sort the dictionary by value descending:
top_100_weighted = sorted(word_counts_weighted.items(), key=lambda x: x[1], reverse=True)[:100]

print("Top 100 words by weighted frequency:")
for word, weight in top_100_weighted:
    print(f"{word}: {weight}")


Top 100 words by weighted frequency:
francophon: 0.9987871304841413
kidal: 0.9987871304841413
macki: 0.9987871304841413
complexion: 0.9987871304841413
everwiden: 0.9987871304841413
pointer: 0.9987871304841413
anteced: 0.9987871304841413
buoyanc: 0.9987871304841413
unachiev: 0.9987871304841413
highwat: 0.9987871304841413
wiriyamu: 0.9987871304841413
chawola: 0.9987871304841413
enthron: 0.9987871304841413
allembrac: 0.9987871304841413
ymca: 0.9987871304841413
footbal: 0.9987871304841413
schoolboy: 0.9987871304841413
newcastl: 0.9987871304841413
knipe: 0.9987871304841413
porter: 0.9987871304841413
phoney: 0.9987871304841413
minc: 0.9987871304841413
bloodier: 0.9987871304841413
quit: 0.9987871304841413
talleyrand: 0.9987871304841413
metaphys: 0.9987871304841413
blameworthi: 0.9987871304841413
infract: 0.9987871304841413
reprob: 0.9987871304841413
thiam: 0.9987871304841413
organizet: 0.9987871304841413
savior: 0.9987871304841413
seaman: 0.9987871304841413
impressionthat: 0.9987871304841413


## Final Cleaning

In [101]:
os.chdir(data_c)
os.chdir(data_freq)

count = joblib.load('word_counts_stemmed.pkl')  # load stemmed counts

# For each speech only keep tokens that appear at least 10x

def select(lista):
    for i in range(len(lista)):
        x = lista[i][0]
        y = lista[i][1]
        y = [w for w in y if count.get(w, 0) >= 10]
        lista[i] = [x, y]
    return lista

for data_path in preprocessed_files:
    data = joblib.load(data_path)
    data = select(data)
    cleaned_path = data_path.replace('.pkl', '_final.pkl')
    joblib.dump(data, cleaned_path)